## Import libraries.

In [2]:
import os                           # For file processing.
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np

In [ ]:
#Set directory values
#split_size=0.1
#print('split_size =',split_size)

# Get the current directory.
current_dir = os.getcwd()

# the data will be written to a working directory.
working_dir    = os.path.join(current_dir,'data')
train_data_dir = os.path.join(working_dir,'Train')
test_data_dir  = os.path.join(working_dir,'Test')
meta_data_dir  = os.path.join(working_dir,'Meta')

print('current_dir      = ',current_dir)
print('working_dir      = ',working_dir)
print('train_data_dir   = ',train_data_dir)
print('test_data_dir    = ',test_data_dir)
print('meta_data_dir    = ',meta_data_dir)
meta_csv_file = (os.path.join(working_dir,'meta.csv'))
print('meta_csv_file    = ',meta_csv_file)

In [7]:
BATCH_SIZE = 64
# The default input size for this model is 224x224.
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
dim = (IMAGE_WIDTH, IMAGE_HEIGHT)

train_generator = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) # VGG16 preprocessing

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing

### Create a training data generator to resize the images to the VGG16 expected size and to split the traininf data into:

In [8]:
print(train_data_dir)
traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

C:\Users\m033\MSc\MachineLearning\IndividualAssignment\data\Train
Found 33337 images belonging to 43 classes.


### Create a training data generator to resize the images to the VGG16 expected size and to split the traininf data into:

In [9]:
validgen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

Found 5872 images belonging to 43 classes.


In [10]:
testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                             class_mode=None,
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

Found 12630 images belonging to 43 classes.


In [12]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [15]:
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes=10

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 50

# First we'll train the model without Fine-tuning
vgg_model = create_model(input_shape, n_classes, optim_1, fine_tune=0)
vgg_model

In [16]:
from livelossplot.inputs.keras import PlotLossesCallback

ModuleNotFoundError: No module named 'livelossplot'

In [11]:


plot_loss_1 = PlotLossesCallback()

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='tl_model_v1.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

ModuleNotFoundError: No module named 'livelossplot'